In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from models import FineTuneBaseModel
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [9]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 6, 5, 17, 27, 52),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [13]:
print(db.query("SELECT POST_ID FROM POSTS WHERE TARGET IN (0, 1)"))

[('nnpn9z',), ('nns6mk',), ('nn4aej',), ('nmpk6k',), ('nmjo2l',), ('nmabst',), ('nm53o9',), ('nmthag',), ('nmpq6d',)]


In [13]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 31, 18, 8, 14))]


In [13]:
db.query('''SELECT TITLE, TARGET FROM POSTS;''')

[('The Gang Buys $GME', -1),
 ('We like the stock $AMC $GME', -1),
 ('Another $GME, $AMC, $BB aimed for the MOON', -1),
 ('Is $SNDL going to do the amc/gme? 🚀', -1),
 ('Lets Go!!! GME to the moon!!!', -1),
 ('GME to the moon!!! And sometimes bananas fuck assholes!!! 💎🙌🦍', -1),
 ("Current Price Action of GME is shockingly similar to the past two major run-ups and 'micro squeezes'",
  -1),
 ('Ape wonders about AMC,GME and BB advertising budget 🙉🚀 SpaceX', -1),
 ("Added $165K more to the GME July 800s YOLO - $646K at risk let's GOOOOO",
  -1),
 ('The day I became a retard. Bought GME pre market at 400+ the day the stocks cratered. Sold Nvidia to',
  -1),
 ('(Part 3) AMC & GME Price Action Comparison before their squeeze', -1)]

In [6]:
db.query('''SELECT COMMENT, TARGET FROM COMMENTS;''')

[('Here before this gets yeeted', -1),
 ('280$ next week just like that guy predicted', -1),
 ('Busy, check back in 2 weeks.', -1),
 ('"TO RISE FOR THE NEXT DECADE"  \n\n\nBitch if it aint rising by next week then I DONT WANT IT.',
  -1),
 ('FUCK YEAH!!!', -1),
 ('Stonk Market Ape Police', -1),
 ("I don't know if I can trust you with a fully charged battery. 🤔", -1),
 ('We want $SNDL  \nWe want $SNDL  \nWe want $SNDL', -1),
 ('Name me a better motivational speech... I dare you....', -1),
 ('Now I have to watch team America again, Trey and Matt are fucking legendary!!!!!',
  -1),
 ("6/9....it's only fitting", -1),
 ('Damn I love GME', -1),
 ('BB is buckling up. Let’s go 🚀🚀🚀', -1),
 ('Amc to the moon! im in and patiently waiting 😃', -1),
 ("This guy has a fully charged phone?!?!?! Something doesn't add up", -1),
 ('Jesus Christ. If there was ever a threat of nuclear war I would take shelter behind your balls. Good luck!',
  -1),
 ('Buy high sell low', -1),
 ('So what’s going to happen?',

In [7]:
db.query("SELECT * FROM STOCKS")

[('GME', datetime.datetime(2021, 6, 5, 17, 25, 53))]

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [12]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

#db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 11, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 6, 5, 17, 28, 22),)]
connection established
[(datetime.datetime(2021, 6, 5, 17, 28, 23),)]


  0%|          | 0/1 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 5, 17, 28, 24),)]


100%|██████████| 1/1 [00:00<00:00,  1.70it/s]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [3]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [ ]:
indices = get_indices("GME")

In [3]:
post_train = PostDataset(512, indices['post_train'])



connection established
[(datetime.datetime(2021, 6, 2, 17, 22, 24),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
sample


{'post': ['GME & AMC',
  'What if GME does a OSTK.',
  'Proper 401k weighting for a 26 year old? 99.8% $GME, 0.2% $AMC'],
 'post_input_ids': tensor([[  101, 14748,  2036,  ...,     0,     0,     0],
         [  101,  1327,  1191,  ...,     0,     0,     0],
         [  101,  5096,  3365,  ...,     0,     0,     0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([[[0., 1.]],
 
         [[1., 0.]],
 
         [[0., 1.]]])}

In [16]:
model = FineTuneBaseModel()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'torch.Tensor'>
tensor([[-0.6015,  0.3814,  0.9997,  ...,  0.9999, -0.6898,  0.9579],
        [-0.6080,  0.4887,  0.9999,  ...,  1.0000, -0.9328,  0.9946],
        [-0.5415,  0.3671,  0.9996,  ...,  0.9998, -0.7365,  0.9712]],
       grad_fn=<TanhBackward>)


In [17]:
out

tensor([[0.7030, 0.2970],
        [0.7028, 0.2972],
        [0.7087, 0.2913]], grad_fn=<SoftmaxBackward>)